In [49]:
import wandb
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [50]:
from image2sphere.pascal_dataset import Pascal3D


path_to_datasets = "/Users/chaykovsky/Downloads/"

dataset = Pascal3D(path_to_datasets, "train")

In [51]:
import sys
import pathlib
import os


# sys.path.append("./image2sphere/src")
# from dataset import ModelNet10Dataset

# if os.path.exists("./clifford-group-equivariant-neural-networks"):
#     original_path = "./clifford-group-equivariant-neural-networks"
#     new_path = "./clifford"
#     os.rename(original_path, new_path)


In [52]:
import sys
import os
# sys.path.append("/kaggle/working/image2sphere")
# original_path = "/kaggle/working/clifford-group-equivariant-neural-networks"
# new_path = "/kaggle/working/clifford"
# os.rename(original_path, new_path)
# sys.path.append("/kaggle/working/clifford")

In [53]:
# !pip install -r /kaggle/working/image2sphere/requirements.txt

In [54]:
# !wget ftp://cs.stanford.edu/cs/cvgl/PASCAL3D+_release1.1.zip

# Imports

## CGENN imports

In [55]:
from clifford.models.modules.linear import MVLinear
from clifford.models.modules.gp import SteerableGeometricProductLayer
from clifford.models.modules.mvlayernorm import MVLayerNorm
from clifford.models.modules.mvsilu import MVSiLU
from clifford.algebra.cliffordalgebra import CliffordAlgebra

In [56]:
algebra = CliffordAlgebra((1, 1, 1, 1, -1))
gp = SteerableGeometricProductLayer(algebra, 1)
x = torch.zeros(1, 1, 32)
x[-1] = 100500
xidx = torch.arange(0, 32, dtype=torch.long)

activ = MVSiLU(algebra=algebra, channels=1)
mv = algebra.embed(x, xidx)
mv = gp(mv)
mv = activ(mv)
mv.size()

torch.Size([1, 1, 32])

In [57]:
from src.model import TralaleroTralala

model = TralaleroTralala(algebra)
model(mv).size()

torch.Size([1, 9, 32])

In [58]:
from image2sphere.models import ResNet
from src.model import MLPBaseline, TralaleroCompetitor

rn = ResNet()
baseline = MLPBaseline()
tralalero = TralaleroCompetitor(algebra)


In [59]:
from src.dataset import PascalSanityCheckDataset
from src.train_utils import train_epoch
from torch.utils.data import DataLoader


dataset = PascalSanityCheckDataset()
loader = DataLoader(dataset, shuffle=True, batch_size=32)

optimizer = torch.optim.Adam(baseline.parameters())
criterion = nn.MSELoss()


In [60]:
mse_baseline = train_epoch(baseline, loader, optimizer, criterion)

100%|██████████| 1/1 [00:06<00:00,  6.81s/it]


In [61]:
dataset = PascalSanityCheckDataset()
loader = DataLoader(dataset, shuffle=True, batch_size=32)

optimizer = torch.optim.Adam(tralalero.parameters())
criterion = nn.MSELoss()

In [62]:
mse_tralalero = train_epoch(tralalero, loader, optimizer, criterion)

100%|██████████| 1/1 [00:06<00:00,  6.64s/it]


In [71]:
print("MSE on 1 batch for MLPBaseline is {}".format(mse_baseline))
print("MSE on 1 batch for Tralalero architecture is {}".format(mse_tralalero))

MSE on 1 batch for MLPBaseline is 0.01835789903998375
MSE on 1 batch for Tralalero architecture is 0.013551775366067886


## I2S imports

In [64]:
import image2sphere.pascal_dataset
from image2sphere.models import ResNet
from image2sphere.pascal_dataset import Pascal3D
from image2sphere.so3_utils import so3_healpix_grid

##  Other imports

In [65]:
import torch
import torch.nn as nn

# CGENN

## CGENN basic blocks

In [66]:
class CGEBlock(nn.Module):
    def __init__(self, algebra, in_features, out_features):
        super().__init__()

        self.layers = nn.Sequential(
            MVLinear(algebra, in_features, out_features),
            MVSiLU(algebra, out_features),
            SteerableGeometricProductLayer(algebra, out_features),
            MVLayerNorm(algebra, out_features)
        )

    def forward(self, input):
        # [batch_size, in_features, 2**d] -> [batch_size, out_features, 2**d]
        return self.layers(input)

class CGEMLP(nn.Module):
    def __init__(self, algebra, in_features, hidden_features, out_features, n_layers=2):
        super().__init__()

        layers = []
        for i in range(n_layers - 1):
            layers.append(
                CGEBlock(algebra, in_features, hidden_features)
            )
            in_features = hidden_features

        layers.append(
            CGEBlock(algebra, hidden_features, out_features)
        )
        self.layers = nn.Sequential(*layers)

    def forward(self, input):
        return self.layers(input)

## CGENN model

In [67]:
class EquvariantCGENN(nn.Module):

    def __init__(self, in_features, hidden_features, out_features):
        super().__init__()
        self.cgemlp = CGEMLP(ca, in_features, hidden_features, hidden_features)
        self.mlp = nn.Sequential(
            nn.Linear(hidden_features, hidden_features),
            nn.ReLU(),
            nn.Linear(hidden_features, hidden_features),
            nn.ReLU(),
            nn.Linear(hidden_features, out_features)
        )

    def forward(self, input):
        h = self.cgemlp(input)
        # Index the hidden states at 0 to get the invariants, and let a regular MLP do the final processing.
        return self.mlp(h[..., 0])

# Final models

In [68]:
class IGA2S(nn.Module):
    def __init__(
        self,
        num_classes, # provided as part of the input
        train_grid_rec_level,
        eval_grid_rec_level,
        size=50,
        pretrained=True,
        global_pooling=True,
        hidden_dim=42
    ):

        self.encoder = ResNet(size, pretrained, global_pooling)

        train_grid = so3.healpix_grid(rec_level=train_grid_rec_level)
        self.register_buffer('train_rotmats', o3.angles_to_matrix(*train_grid))


        eval_grid = so3.healpix_grid(rec_level=eval_grid_rec_level)
        self.register_buffer('eval_rotmats', o3.angles_to_matrix(*eval_grid))

        input_dim = self.encoder.output_shape[0]
        self.equiv = EquvariantCGENN(input_dim + num_classes, hidden_dim, )


## Model 1: ResNet + CGENN head

# Training

## Data prep

## Training & Evaluation functions

## Training